In [1]:
import theano.tensor as tt
import sympy as sym
import pymc3 as pm
import numpy as np
from sunode.wrappers.as_theano import solve_ivp

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


In [4]:
def switch_zero_sym(t, t1):
    return 1 / (1 + sym.exp(5 * (t - t1)))


def switch_eta_sym(t, t1, eta, nu, xi):
    return eta + (1 - eta) / (1 + sym.exp(xi * (t - t1 - nu)))


def seir_rhs(t, y, p):
    # Total number of infectious people
    nI = sum(y.I)
    ntot = sum(y.S + y.E + y.I + y.A)
    p_tmax = switch_zero_sym(t, p.tmax2)
    p_tswitch = switch_eta_sym(t, p.tswitch, p.eta, p.nu, p.xi)
    
    return {
        'S': -p.beta * p_tswitch * y.S * nI / ntot,
        'E': (
            p.beta * p_tswitch * y.S * nI / ntot
            - 1 / p.inv_tau * p_tmax * y.E
        ),
        'I': (
            p.psi / p.inv_tau * p_tmax * y.E
            - 1 / p.inv_mu * y.I
        ),
        'A': (
            (1 - p.psi) / p.inv_tau * p_tmax * y.E
            - 1 / p.inv_mu * y.A
        ),
        'R': (
            1 / p.inv_mu * (y.I - y.A)
        ),
        'C': p.psi / p.inv_tau * p_tmax * y.E
    }

In [12]:
# TODO These numbers are just placeholders for now
n_ages = 10

default_priors = [
    ('beta', pm.Exponential, {'lam': 1}),
    ('eta', pm.Beta, {'alpha': 1, 'beta': 1}),
    ('epsilon', pm.Beta, {'alpha': 1, 'beta': 1, 'shape': n_ages}),
    ('rho', pm.Beta, {'alpha': 1, 'beta': 1, 'shape': n_ages - 1}),
    ('pi', pm.Beta, {'alpha': 1, 'beta': 1}),
    ('phi', pm.Exponential, {'lam': 1}),
    ('xi_raw', pm.Beta, {'alpha': 1, 'beta': 1}),
    ('nu', pm.Exponential, {'lam': 1}),
]

default_ode_params = {
    'inv_tau': np.array(1.),
    'inv_mu': np.array(1.),
    'psi': np.array(1.),
    'tmax2': np.array(1.),
    'tswitch': np.array(1.),
}

age_dist = np.ones(n_ages)
t0 = 0
tvals = np.linspace(0, 1000)
pop_t = 1
observed_slice = slice(500, None)

In [13]:
prior_overrides = {
    'beta': {'lam': 2}
}

ode_params_overrides = {}

In [14]:
class ModelConfig:
    pass

In [15]:
# TODO Wrap in make_model function
with pm.Model() as model:
    priors = {}
    for name, dist, params in default_priors:
        params = params.copy()
        params.update(prior_overrides.get(name, {}))
        rv = dist(name, **params)
        priors[name] = rv

    xi = priors['xi_raw'] + 0.5
    rho_K = tt.concatenate([priors['rho'], [1]])

    y0 = {
        'S': (age_dist * (1 - priors['pi']), (n_ages,)),
        'E': (age_dist * priors['pi'], (n_ages,)),
        'I': np.zeros(n_ages),
        'A': np.zeros(n_ages),
        'R': np.zeros(n_ages),
        'C': np.zeros(n_ages),
    }

    params = {
        name: (priors[name], ())
        for name in ['beta', 'eta', 'nu']
    }
    params['xi'] = (xi, ())

    ode_params = default_ode_params.copy()
    ode_params.update(ode_params_overrides)
    params.update({
        name: (val, ())
        for name, val in ode_params.items()
    })

    y, flat_solution, problem, solver, *_ = solve_ivp(
        t0=t0,
        y0=y0,
        params=params,
        tvals=tvals,
        rhs=seir_rhs,
        solver_kwargs={
            'constraints': np.ones(6 * n_ages),
        },
    )

    for name in y:
        y[name] = pop_t * y[name]
    
    diff_C = tt.concatenate([
        y['C'][:1, :],
        y['C'][:-1, :] - y['C'][1:, :]],
    )
    
    output_incidence_cases = tt.dot(diff_C[observed_slice], rho_K)
    

    output = ModelConfig()
    output.model = model
    output.problem = problem
    output.solver = solver
    #return output  # TODO Save stuff in output